In [ ]:
!pip install gliner torch transformers seqeval evaluate corus --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.2 MB/s eta 0:00:00
   ━

In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip
!rm collection5.zip
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

--2025-04-16 15:22:57--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.07MB/s    in 1.7s    

2025-04-16 15:23:00 (1.07 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection

In [ ]:
!unzip merged_dataset.zip -d ./

Archive:  merged_dataset.zip
   creating: ./merged_dataset/
  inflating: ./merged_dataset/dataset_dict.json  
   creating: ./merged_dataset/train/
  inflating: ./merged_dataset/train/data-00000-of-00001.arrow  
  inflating: ./merged_dataset/train/state.json  
  inflating: ./merged_dataset/train/dataset_info.json  
   creating: ./merged_dataset/test/
  inflating: ./merged_dataset/test/data-00000-of-00001.arrow  
  inflating: ./merged_dataset/test/state.json  
  inflating: ./merged_dataset/test/dataset_info.json  


In [ ]:
from functools import partial
import os
import random
import re
from typing import Any, Dict, Callable, List, Tuple

from corus import load_ne5, load_lenta
from corus.sources.lenta import LentaRecord
from datasets import concatenate_datasets, Dataset, DatasetDict, ClassLabel, load_from_disk
import evaluate
from gliner import GLiNER
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    AutoModelForTokenClassification,
    DataCollatorForLanguageModeling,
    DataCollatorForTokenClassification,
    EarlyStoppingCallback,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments,
)
import torch
import torch.nn as nn

In [ ]:
def tokenize_text(text: str) -> Tuple[List[str], List[Tuple[int, int]]]:
    """
    Токенизация текста с сохранением смещений.
    Используем регулярное выражение, чтобы разбить текст на слова и знаки пунктуации.
    Возвращает список токенов и соответствующий список кортежей (start, end) для каждого токена.
    """
    # Регулярное выражение для слов, чисел и знаков препинания
    pattern = r"\w+|[^\w\s]"
    tokens = []
    offsets = []
    for match in re.finditer(pattern, text):
        tokens.append(match.group())
        offsets.append((match.start(), match.end()))
    return tokens, offsets


def load_from_corus(path: str = "./Collection5") -> List[Dict[str, Any]]:
    records = load_ne5(path)
    dataset = []
    for record in records:
        text = record.text
        spans = record.spans
        tokens, offsets = tokenize_text(text)
        for span in spans:
            ent_start = span.start
            ent_end = span.stop
            ent_label = span.type
            token_labels = ["O"] * len(tokens)
            for i, (token_start, token_end) in enumerate(offsets):
                if token_end <= ent_start or token_start >= ent_end:
                    continue
                if token_start == ent_start:
                    token_labels[i] = f"B-{ent_label}"
                else:
                    token_labels[i] = f"I-{ent_label}"
        dataset.append({"text": text, "tokens": tokens, "ner_tags": token_labels})
    return dataset


def get_unique_tags(raw_ner_dataset: List[Dict[str, Any]]) -> List[str]:
    unique_tags = set()
    for rec in raw_ner_dataset:
        for tag in rec["ner_tags"]:
            unique_tags.add(tag)
    return list(unique_tags)


def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels


def tokenize_and_align_labels(examples: Dict[str, Any], tokenizer: PreTrainedTokenizer):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs


def get_class_weights(dataset: Dataset, id2label: Dict[int, str]) -> torch.Tensor:
    label_counts = []
    for example in dataset:
        label_counts += [label for label in example["ner_tags"] if label != -100]
    labels = np.array(label_counts)
    class_weights = compute_class_weight(
        class_weight="balanced", classes=np.unique(labels), y=labels
    )
    weights = torch.tensor(class_weights, dtype=torch.float)
    return weights


def process_ner_dataset(
    raw_ner_dataset: List[Dict[str, Any]], id2label: Dict[int, str], label2id: Dict[str, int], model_id: str
) -> Tuple[Dataset, Dict[int, str], torch.Tensor]:
    raw_ner_dataset = load_from_corus()
    unique_tags = list(sorted(get_unique_tags(raw_ner_dataset), reverse=True))
    dataset = Dataset.from_list(raw_ner_dataset)
    label = ClassLabel(names=unique_tags)
    dataset = dataset.map(
        lambda x: {"ner_tags": [label2id[tag] for tag in x["ner_tags"]]}
    )
    class_weights = get_class_weights(dataset, id2label)
    dataset = dataset.map(
        partial(
            tokenize_and_align_labels, tokenizer=AutoTokenizer.from_pretrained(model_id)
        ),
        batched=True,
        remove_columns=dataset.column_names,
    )
    return dataset, class_weights


def split_dataset(dataset: Dataset, test_size: float, seed: int) -> DatasetDict:
    return dataset.train_test_split(test_size=test_size, shuffle=True, seed=seed)

In [ ]:
def mlm_preprocess_function(
    examples: Dict[str, Any], tokenizer: PreTrainedTokenizer
) -> List[str]:
    return tokenizer(examples["text"])


def group_texts(examples: Dict[str, Any], block_size: int) -> List[str]:
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result


def process_mlm_dataset(
    raw_dataset: List[Dict[str, str]], model_id: str, block_size: int
) -> Dataset:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    dataset = Dataset.from_list(raw_dataset)
    dataset = dataset.map(
        partial(mlm_preprocess_function, tokenizer=tokenizer),
        batched=True,
        remove_columns=dataset.column_names,
    )
    dataset = dataset.map(
        partial(group_texts, block_size=block_size),
        batched=True,
    )
    return dataset

In [ ]:
class WeightedLossTrainer(Trainer):
    def __init__(self, class_weights: torch.Tensor, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(kwargs.get("model").device)
        self.loss_fct = nn.CrossEntropyLoss(
            weight=self.class_weights, ignore_index=-100
        )

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        inputs = inputs.to(model.device)
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss = self.loss_fct(logits.view(-1, logits.shape[-1]), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


In [ ]:
def get_ner_args(
    output_dir: str, num_epochs: int = 100, batch_size: int = 64
) -> TrainingArguments:
    return TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        remove_unused_columns=False,
        eval_strategy="steps",
        eval_steps=0.05,
        logging_strategy="steps",
        logging_steps=0.01,
        save_strategy="steps",
        save_steps=0.2,
        save_total_limit=3,
        greater_is_better=True,
        num_train_epochs=num_epochs,
        metric_for_best_model="f1",
        learning_rate=4e-5,
        weight_decay=0.01,
        warmup_ratio=0.06,
        report_to=[]
    )


def compute_ner_metrics(
    eval_preds: Tuple[np.ndarray, np.ndarray],
    id2label: Dict[int, str],
    metrics: Callable = evaluate.load("seqeval"),
) -> Dict[str, float]:
    logits, gts = eval_preds
    preds = np.argmax(logits, axis=-1)

    true_labels = [[id2label[l] for l in gt if l != -100] for gt in gts]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, gts)
    ]
    all_metrics = metrics.compute(
        predictions=true_predictions, references=true_labels, zero_division=0.0
    )
    return {
        "precision": float(all_metrics["overall_precision"]),
        "recall": float(all_metrics["overall_recall"]),
        "f1": float(all_metrics["overall_f1"]),
        "accuracy": float(all_metrics["overall_accuracy"]),
    }


def get_ner_trainer(
    model_id: str,
    output_dir: str,
    dataset: Dataset,
    id2label: Dict[int, str],
    class_weights: torch.Tensor,
    early_stopping_patience: int = 10,
) -> None:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForTokenClassification.from_pretrained(
        model_id, num_labels=len(id2label)
    )
    trainer = WeightedLossTrainer(
        model=model,
        args=get_ner_args(output_dir),
        class_weights=class_weights,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        processing_class=tokenizer,
        compute_metrics=partial(compute_ner_metrics, id2label=id2label),
        callbacks=[
            EarlyStoppingCallback(early_stopping_patience=early_stopping_patience),
        ],
    )
    return trainer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def get_mlm_args(
    output_dir: str, num_epochs: int = 50, batch_size: int = 64
) -> TrainingArguments:
    return TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy="steps",
        logging_steps=0.01,
        save_strategy="steps",
        save_steps=0.2,
        save_total_limit=3,
        remove_unused_columns=False,
        num_train_epochs=num_epochs,
        learning_rate=2e-5,
        weight_decay=0.01,
        report_to=[]
    )


def get_mlm_trainer(
    model_id: str,
    output_dir: str,
    dataset: DatasetDict,
) -> Trainer:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForMaskedLM.from_pretrained(model_id)
    args = get_mlm_args(output_dir)
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer, mlm=True, mlm_probability=0.15
        ),
        processing_class=tokenizer,
    )
    return trainer


In [ ]:
def get_entities(
    examples: Dict[str, Any],
    model: GLiNER,
    tokenizer: PreTrainedTokenizer,
    labels: List[str],
    labels_encoder: Dict[str, str],
) -> List[List[Dict[str, Any]]]:
    texts = examples["text"]
    examples["ner_tags"] = []
    batch_entities = model.batch_predict_entities(texts, labels, threshold=0.4)
    for text, entities in zip(texts, batch_entities):
        tokens, token_offsets = tokenize_text(text)
        bio_tags = ["O"] * len(tokens)
        for entity in entities:
            ent_start = entity["start"]
            ent_end = entity["end"]
            ent_label = entity["label"]

            for idx, (start, end) in enumerate(token_offsets):
                if start >= ent_start and end <= ent_end:
                    if start == ent_start:
                        bio_tags[idx] = f"B-{labels_encoder[ent_label]}"
                    else:
                        bio_tags[idx] = f"I-{labels_encoder[ent_label]}"

        examples["ner_tags"].append(bio_tags)
    return examples


def get_additional_dataset(
    label_model_id: str,
    train_model_id: str,
    labels: List[str] = ["person", "location", "organization", "media", "geopolitical"],
    labels_encoder: Dict[str, str] = {
        "person": "PER",
        "location": "LOC",
        "organization": "ORG",
        "media": "MEDIA",
        "geopolitical": "GEOPOLIT",
    },
) -> Dataset:
    records = load_from_corus()
    dataset = Dataset.from_list([{"text": item["text"]} for item in records])
    rand_idx = random.choices(range(len(dataset)), k=1000)
    dataset = dataset.select(rand_idx)
    tokenizer = AutoTokenizer.from_pretrained(train_model_id)
    model = GLiNER.from_pretrained(label_model_id, max_length=512)
    dataset = dataset.map(
        partial(
            get_entities,
            model=model,
            labels=labels,
            tokenizer=tokenizer,
            labels_encoder=labels_encoder,
        ),
        batched=True,
        batch_size=16,
    )
    return dataset


def load_from_lenta(path: str = "./lenta-ru-news.csv.gz") -> List[LentaRecord]:
    records = load_lenta(path)
    return records

In [ ]:
rubert_tiny_model_id = "cointegrated/rubert-tiny2"
pretrained_mlm_rubert_path = "./rubert_tiny_mlm"
ner_mlm_rubert_path = "./rubert_tiny_pretrain_mlm_ner"
gliner_model_id = "urchade/gliner_multi-v2.1"
merged_dataset_path = "./merged_dataset"
ID2LABEL = {
    0: "O",
    1: "B-GEOPOLIT",
    2: "B-LOC",
    3: "B-MEDIA",
    4: "B-ORG",
    5: "B-PER",
    6: "I-GEOPOLIT",
    7: "I-LOC",
    8: "I-MEDIA",
    9: "I-ORG",
    10: "I-PER",
}
LABEL2ID = {
    "O": 0,
    "B-GEOPOLIT": 1,
    "B-LOC": 2,
    "B-MEDIA": 3,
    "B-ORG": 4,
    "B-PER": 5,
    "I-GEOPOLIT": 6,
    "I-LOC": 7,
    "I-MEDIA": 8,
    "I-ORG": 9,
    "I-PER": 10,
}

## Все умозаключения и выборы гиперпараметров обосновываются здесь:
- Предобработка не делается, т.к. используем трансформер модели
- Для борьбы с преобладающим большинством O меток (как выясняется при обучении в лоб) добавлены веса классам в cross entropy loss
- Гиперпараметры оптимизаторов и обучения взяты из рабочей практики и нескольких запусков обучения
- Для объединенного датасета использовалось только 1000 объектов, т.к. посчитать больше на моем устройстве сложно + я использовал few-shot модель для NER из-за чего inference намного дольше

In [ ]:
raw_ner_dataset = load_from_corus()
raw_mlm_dataset = [{"text": item["text"]} for item in raw_ner_dataset]
dataset, class_weights = process_ner_dataset(
    raw_ner_dataset, ID2LABEL, LABEL2ID, rubert_tiny_model_id
)
dataset = split_dataset(dataset, 0.3, 42)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer_rubert = get_ner_trainer(
    rubert_tiny_model_id,
    "./rubert_tiny_ner",
    dataset,
    ID2LABEL,
    class_weights,
    early_stopping_patience=10,
)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
before_train_rubert = trainer_rubert.evaluate()
before_train_rubert

{'eval_loss': 2.5638976097106934,
 'eval_model_preparation_time': 0.0014,
 'eval_precision': 0.0006555271642429999,
 'eval_recall': 0.11085972850678733,
 'eval_f1': 0.0013033474750967537,
 'eval_accuracy': 0.012921774981013346,
 'eval_runtime': 7.0293,
 'eval_samples_per_second': 42.678,
 'eval_steps_per_second': 0.711}

In [15]:
trainer_rubert.train()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
55,1.408200,1.351589,0.001400,0.000000,0.000000,0.000000,0.000781


Step,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
55,1.408200,1.351589,0.001400,0.000000,0.000000,0.000000,0.000781
110,0.851000,0.920149,0.001400,0.002978,0.194570,0.005866,0.529576
165,0.533500,0.731388,0.001400,0.007156,0.407240,0.014065,0.645449
220,0.328900,0.684654,0.001400,0.009926,0.497738,0.019464,0.667115
275,0.200600,0.726767,0.001400,0.012607,0.531674,0.024631,0.706314
330,0.136600,0.726475,0.001400,0.016119,0.572398,0.031355,0.750472
385,0.098900,0.770415,0.001400,0.018872,0.595023,0.036584,0.780840
440,0.076200,0.811337,0.001400,0.020908,0.595023,0.040396,0.800944
495,0.062900,0.918879,0.001400,0.024506,0.583710,0.047037,0.828361
550,0.053300,0.904939,0.001400,0.026018,0.592760,0.049848,0.836324


TrainOutput(global_step=1100, training_loss=0.26024747742847965, metrics={'train_runtime': 2565.5488, 'train_samples_per_second': 27.285, 'train_steps_per_second': 0.429, 'total_flos': 1240543227752184.0, 'train_loss': 0.26024747742847965, 'epoch': 100.0})

In [16]:
after_train_rubert = trainer_rubert.evaluate()
after_train_rubert

{'eval_loss': 1.1489518880844116,
 'eval_model_preparation_time': 0.0014,
 'eval_precision': 0.03862660944206009,
 'eval_recall': 0.5497737556561086,
 'eval_f1': 0.07218179117778109,
 'eval_accuracy': 0.9020180101985462,
 'eval_runtime': 6.0348,
 'eval_samples_per_second': 49.712,
 'eval_steps_per_second': 0.829,
 'epoch': 100.0}

In [17]:
mlm_dataset = process_mlm_dataset(raw_mlm_dataset, rubert_tiny_model_id, 128)
mlm_dataset = split_dataset(mlm_dataset, 0.3, 42)
mlm_trainer = get_mlm_trainer(rubert_tiny_model_id, pretrained_mlm_rubert_path, mlm_dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2394 > 2048). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [18]:
mlm_trainer.train()

Step,Training Loss
14,3.620800
28,3.559600
42,3.539000
56,3.533300
70,3.424600
84,3.437600
98,3.425900
112,3.360600
126,3.339500
140,3.378900


TrainOutput(global_step=1350, training_loss=3.0951822648225007, metrics={'train_runtime': 880.118, 'train_samples_per_second': 97.43, 'train_steps_per_second': 1.534, 'total_flos': 163605007104000.0, 'train_loss': 3.0951822648225007, 'epoch': 50.0})

In [19]:
mlm_trainer.save_model(pretrained_mlm_rubert_path)

In [20]:
trainer_rubert = get_ner_trainer(
    pretrained_mlm_rubert_path,
    "./rubert_tiny_pretrain_mlm_ner",
    dataset,
    ID2LABEL,
    class_weights,
    early_stopping_patience=10,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ./rubert_tiny_mlm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
trainer_rubert.evaluate()

{'eval_loss': 2.4418656826019287,
 'eval_model_preparation_time': 0.0017,
 'eval_precision': 0.00026101823212351384,
 'eval_recall': 0.04524886877828054,
 'eval_f1': 0.0005190423668331928,
 'eval_accuracy': 0.028935662363024844,
 'eval_runtime': 4.9951,
 'eval_samples_per_second': 60.058,
 'eval_steps_per_second': 1.001}

In [26]:
!zip rubert-tiny-ner -r ./rubert_tiny_ner


updating: rubert_tiny_ner/ (stored 0%)
updating: rubert_tiny_ner/checkpoint-660/ (stored 0%)
updating: rubert_tiny_ner/checkpoint-660/trainer_state.json (deflated 79%)
updating: rubert_tiny_ner/checkpoint-660/config.json (deflated 57%)
updating: rubert_tiny_ner/checkpoint-660/vocab.txt (deflated 64%)
updating: rubert_tiny_ner/checkpoint-660/scheduler.pt (deflated 55%)
updating: rubert_tiny_ner/checkpoint-660/special_tokens_map.json (deflated 80%)
updating: rubert_tiny_ner/checkpoint-660/tokenizer_config.json (deflated 73%)
updating: rubert_tiny_ner/checkpoint-660/optimizer.pt (deflated 65%)
updating: rubert_tiny_ner/checkpoint-660/rng_state.pth (deflated 25%)
updating: rubert_tiny_ner/checkpoint-660/training_args.bin (deflated 51%)
updating: rubert_tiny_ner/checkpoint-660/model.safetensors (deflated 8%)
updating: rubert_tiny_ner/checkpoint-660/tokenizer.json (deflated 73%)
updating: rubert_tiny_ner/checkpoint-880/ (stored 0%)
updating: rubert_tiny_ner/checkpoint-880/trainer_state.json 

In [24]:
!zip rubert-tiny-mlm -r ./rubert_tiny_mlm


  adding: rubert_tiny_mlm/ (stored 0%)
  adding: rubert_tiny_mlm/config.json (deflated 48%)
  adding: rubert_tiny_mlm/vocab.txt (deflated 64%)
  adding: rubert_tiny_mlm/checkpoint-1080/ (stored 0%)
  adding: rubert_tiny_mlm/checkpoint-1080/trainer_state.json (deflated 81%)
  adding: rubert_tiny_mlm/checkpoint-1080/config.json (deflated 48%)
  adding: rubert_tiny_mlm/checkpoint-1080/vocab.txt (deflated 64%)
  adding: rubert_tiny_mlm/checkpoint-1080/scheduler.pt (deflated 56%)
  adding: rubert_tiny_mlm/checkpoint-1080/special_tokens_map.json (deflated 80%)
  adding: rubert_tiny_mlm/checkpoint-1080/tokenizer_config.json (deflated 73%)
  adding: rubert_tiny_mlm/checkpoint-1080/optimizer.pt (deflated 7%)
  adding: rubert_tiny_mlm/checkpoint-1080/rng_state.pth (deflated 25%)
  adding: rubert_tiny_mlm/checkpoint-1080/training_args.bin (deflated 51%)
  adding: rubert_tiny_mlm/checkpoint-1080/model.safetensors (deflated 8%)
  adding: rubert_tiny_mlm/checkpoint-1080/tokenizer.json (deflated 73%)

In [25]:
trainer_rubert.train()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
if not os.path.exists(merged_dataset_path):
    labeled_news_dataset = get_additional_dataset(gliner_model_id, rubert_tiny_model_id)
    ner_dataset = Dataset.from_list(raw_ner_dataset)
    raw_merged_dataset = concatenate_datasets([ner_dataset, labeled_news_dataset])
    merged_dataset, class_weights = process_ner_dataset(
        raw_merged_dataset, ID2LABEL, LABEL2ID, rubert_tiny_model_id
    )
    merged_dataset = split_dataset(raw_merged_dataset, 0.3, 42)
    merged_dataset.save_to_disk(merged_dataset_path)
merged_dataset = load_from_disk(f"{merged_dataset_path}")

In [ ]:
trainer_rubert = get_ner_trainer(
    rubert_tiny_model_id,
    "./rubert_tiny_merged_ner",
    merged_dataset,
    ID2LABEL,
    class_weights,
    early_stopping_patience=10,
)

In [ ]:
trainer_rubert.evaluate()

In [ ]:
trainer_rubert.train()

In [ ]:
trainer_rubert.evaluate()

Финально посмотрим на classification_report для каждой из моделей